In [2]:
import sys
import os

# Thêm thư mục gốc (project/) vào sys.path
# Nếu __file__ không có (trong notebook), thay bằng:
sys.path.append(os.path.abspath(".."))

from services.products import ProductServices


In [3]:
from fastapi import APIRouter, HTTPException
from pydantic import BaseModel
import os, json, dotenv
from autogen import AssistantAgent, ConversableAgent
import uuid
from db import Session
from models.fqas import FQA
from repositories.message import MessageRepository
from controllers.search import search
import traceback
from models.message import MessageModel, CreateMessagePayload
from autogen import register_function
from env import env

In [4]:

config_list = [{
    "model": "gemini-2.0-flash",
    "api_key": env.GEMINI_API_KEY,
    "api_type": "google"
}]

class ChatbotRequest(BaseModel):
    chat_id: int
    message: str

In [5]:
Manager = ConversableAgent(
    name="manager",
    system_message="""Bạn là một trợ lý AI thông minh làm việc cho một sàn thương mại điện tử IUH-Ecomerce
    Bạn sẽ nhận đầu vào câu hỏi của người dùng về sàn thương mại điện tử IUH-Ecomerce
    Nhiệm vụ của bạn là trả lời câu hỏi của người dùng một cách chính xác và đầy đủ nhất có thể
    Nếu bạn chưa đủ thông tin trả lời, bạn hãy sử dụng các trợ lý khác để tìm kiếm thông tin
    Hãy trả về mô tả truy vấn Qdrant dưới dạng JSON:"
        "agent": "ProductAgent" | "PoliciAgent" | "MySelf" | "TransactionAgent" ,
        "query": String
    Với Agent là tên của trợ lý mà bạn muốn sử dụng để tìm kiếm thông tin
        Trong đó ProductAgent là trợ lý tìm kiếm thông tin sản phẩm
        Trong đó PoliciAgent là trợ lý tìm kiếm thông tin chính sách
        Trong đó MySelf là trợ lý tìm trả lời câu hỏi bình thường
        Trong đó TransactionAgent là trợ lý tìm kiếm thông tin giao dịch


        """,    

        
    llm_config={"config_list": config_list},
    human_input_mode= "NEVER"
)

In [14]:
async def get_product_info(query):
    chat = await Manager.a_generate_reply(
    messages=[{"role": "user", "content": query}])
    # response= json.loads(chat)
    return chat["content"]



In [28]:
import re
def extract_qdrant_query(response: str):
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', response, re.DOTALL) or \
                    re.search(r'(\{.*?\})', response, re.DOTALL)
    if not json_match:
        return {"collection_name": "products", "payload": "", "limit": 5}
    try:
        return json.loads(json_match.group(1))
    except json.JSONDecodeError as e:
        return {"collection_name": "products", "payload": "", "limit": 5}

In [31]:
returm = await get_product_info("hãy nói về chính sách bảo hành của IUH-Ecommerce")
returm = extract_qdrant_query(returm)
returm

{'agent': 'PoliciAgent', 'query': 'Chính sách bảo hành của IUH-Ecommerce'}

In [97]:
from controllers.qdrant_agent import chatbot_endpoint as product_agent
from controllers.polici_agent import ask_chatbot as policy_agent

In [100]:
### call agent
ChatbotReques1t = {
    "chat_id": 2,
    "message": "sách nói về dao dịch "
}


async def call_agent(agent, ChatbotRequest : ChatbotRequest): 
    if agent == "ProductAgent":
        # Gọi hàm tìm kiếm sản phẩm
        result = await product_agent(ChatbotRequest)
        return result
    elif agent == "PoliciAgent":
        # Gọi hàm tìm kiếm chính sách
        result = policy_agent(ChatbotRequest)
        return result["response"]
    elif agent == "MySelf":
        # Gọi hàm tìm kiếm thông tin cá nhân
        result = await search(ChatbotRequest['message'])
        return result
    elif agent == "TransactionAgent":
        # Gọi hàm tìm kiếm thông tin giao dịch
        result = await search(ChatbotRequest['message'])
        return result





In [101]:
## test
response = await get_product_info("chinh sách đổi trả hàng")
class ChatbotRequest(BaseModel):
    chat_id: int
    message: str
ChatbotRequest = ChatbotRequest(
    chat_id=2,
    message=response["query"]
)

await call_agent(response["agent"], ChatbotRequest)

User (to Assistant):

Người dùng hỏi: chính sách đổi trả hàng

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


/home/huong/.cache/pypoetry/virtualenvs/chatbot-k9VqB5fJ-py3.12/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:3424: UserWarning: Function 'search' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/home/huong/.cache/pypoetry/virtualenvs/chatbot-k9VqB5fJ-py3.12/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:3322: UserWarning: Function 'search' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


Assistant (to User):

***** Suggested tool call (call_r7ZqzflDVfgVZ6Bv9xjh6jrS): search *****
Arguments: 
{"payload":"chính sách đổi trả hàng","collection_name":"poli_embeddings"}
***********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search...
Call ID: call_r7ZqzflDVfgVZ6Bv9xjh6jrS
Input arguments: {'payload': 'chính sách đổi trả hàng', 'collection_name': 'poli_embeddings'}
User (to Assistant):

***** Response from calling tool (call_r7ZqzflDVfgVZ6Bv9xjh6jrS) *****
Đảm bảo rằng sản phẩm của quý khách đủ điều kiện theo chính sách của IUH-Ecomerce
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

Chính sách đổi trả hàng của IUH-Ecomerce yêu cầu sản phẩm của quý khách phải đủ điều kiện theo quy định. Quý khách

'Chính sách đổi trả hàng của IUH-Ecomerce yêu cầu sản phẩm của quý khách phải đủ điều kiện theo quy định. Quý khách có thể thực hiện việc đổi trả trong thời gian quy định và cần tuân thủ các hướng dẫn cụ thể để đảm bảo quá trình diễn ra thuận lợi.'

In [102]:
## endpoint
@router.post("/chat")
async def chat(request: ChatbotRequest):
    try:
        # Gọi hàm tìm kiếm sản phẩm
        result = await get_product_info(request.message)
        response = await call_agent(result["agent"], request)
        return response
    except Exception as e:
        print("Error:", e)
        raise HTTPException(status_code=500, detail="Internal Server Error")

FastAPIError: Invalid args for response field! Hint: check that chat_id=2 message='chính sách đổi trả hàng' is a valid Pydantic field type. If you are using a return type annotation that is not a valid Pydantic field (e.g. Union[Response, dict, None]) you can disable generating the response model from the type annotation with the path operation decorator parameter response_model=None. Read more: https://fastapi.tiangolo.com/tutorial/response-model/